In [1]:
import pandas as pd
import numpy as np
import geopandas as gpd
import os
import datetime
import matplotlib.pyplot as plt
import contextily as cx

## 读数据

In [2]:
path = 'process data/'
df = pd.read_csv(path + 'dataset_3_raw.csv')
df

,device_id,latitude,longitude,date
0,f664b4d6-5809-413e-b49b-3605d0515503,22.338962,114.207170,2023-09-04 07:41:03
1,e9017f7c-c52f-4a6a-926e-b1c0976b1248,22.288587,114.195222,2023-09-05 11:16:02
2,f6a9d8b6-5e2c-488f-9ee7-f58e97e5d5de,22.340000,114.200000,2023-09-14 11:21:56
3,ff22c280-126f-600f-214c-0ab0411394db,22.370519,114.132008,2023-09-01 18:15:22
4,ff22c280-126f-600f-214c-0ab0411394db,22.368826,114.118049,2023-09-01 20:50:06
...,...,...,...,...
3129760,b58ded3e-5a73-47e9-8af1-58ba760ac6d0,22.395124,113.974522,2023-09-06 06:47:37
3129761,b58ded3e-5a73-47e9-8af1-58ba760ac6d0,22.395138,113.974516,2023-09-06 07:15:54
3129762,b5933b4e-0e52-4398-8253-1f27bb4bec07,22.273868,114.187822,2023-09-15 03:06:44
3129763,b460a842-0630-4ebf-963b-6285b81f5574,22.310000,114.230000,2023-09-14 13:08:46


In [5]:
import datetime
df['date'] = pd.to_datetime(df['date'])

In [9]:
## 没有home经纬度的数据中，每个整点的数据量
df['date'].dt.hour.value_counts().sort_index()

0      61483
1      51411
2      55326
3      57644
4      45765
5      47723
6      85847
7     181680
8      91768
9     171672
10    204694
11    217428
12    239160
13    249172
14    218030
15    208176
16    177224
17    149450
18    148392
19    100611
20     88213
21     99076
22     99682
23     80138
Name: date, dtype: int64

In [6]:
df_15 = df[(df['date'].dt.hour >= 1) & (df['date'].dt.hour <= 5)] # 凌晨1：00至3：59的数据
df_15 = df_15[['device_id','latitude','longitude']]
df_15

,device_id,latitude,longitude
36,e60fbeb4-7e78-4985-bcc3-09ff874cb7e3,22.445384,114.035978
37,e60fbeb4-7e78-4985-bcc3-09ff874cb7e3,22.445352,114.035938
80,eb99111a-261c-4456-8da3-9f14c043b571,22.448940,113.998410
81,eb99111a-261c-4456-8da3-9f14c043b571,22.448940,113.998430
82,eb99111a-261c-4456-8da3-9f14c043b571,22.448920,113.998430
...,...,...,...
3129717,afc663ee-9b82-4bfc-a45f-6cce84cb35b2,22.424879,114.229265
3129718,afc663ee-9b82-4bfc-a45f-6cce84cb35b2,22.424917,114.229258
3129719,afc663ee-9b82-4bfc-a45f-6cce84cb35b2,22.424910,114.229247
3129720,afc663ee-9b82-4bfc-a45f-6cce84cb35b2,22.424897,114.229254


In [10]:
device_cnt = np.size(df_15['device_id'].unique())
device_cnt
## 可以推测住址的用户有 44258 个

44258

In [13]:
## 转换为 GeoDataFrame
geometry = gpd.points_from_xy(df_15['longitude'], df_15['latitude'])
location_lat_lon_gdf = gpd.GeoDataFrame(df_15, geometry=geometry)
location_lat_lon_gdf.crs = 'EPSG:4326'
location_lat_lon_gdf = location_lat_lon_gdf.drop(['longitude','latitude'], axis=1)
location_lat_lon_gdf

,device_id,geometry
36,e60fbeb4-7e78-4985-bcc3-09ff874cb7e3,POINT (114.03598 22.44538)
37,e60fbeb4-7e78-4985-bcc3-09ff874cb7e3,POINT (114.03594 22.44535)
80,eb99111a-261c-4456-8da3-9f14c043b571,POINT (113.99841 22.44894)
81,eb99111a-261c-4456-8da3-9f14c043b571,POINT (113.99843 22.44894)
82,eb99111a-261c-4456-8da3-9f14c043b571,POINT (113.99843 22.44892)
...,...,...
3129717,afc663ee-9b82-4bfc-a45f-6cce84cb35b2,POINT (114.22927 22.42488)
3129718,afc663ee-9b82-4bfc-a45f-6cce84cb35b2,POINT (114.22926 22.42492)
3129719,afc663ee-9b82-4bfc-a45f-6cce84cb35b2,POINT (114.22925 22.42491)
3129720,afc663ee-9b82-4bfc-a45f-6cce84cb35b2,POINT (114.22925 22.42490)


## 读building shape file

In [14]:
shapefile = gpd.read_file('Building_SHP/BUILDING_STRUCTURE.shp', encoding="GBK")
shapefile = shapefile.to_crs('EPSG:4326')
building_shp = shapefile[shapefile['BUILDING00'] == 'T']
building_shp = building_shp[['BUILDINGST', 'geometry']]
building_shp

,BUILDINGST,geometry
0,243561,"POLYGON ((113.84091 22.21870, 113.84090 22.218..."
1,243562,"POLYGON ((113.84094 22.21863, 113.84095 22.218..."
2,243563,"POLYGON ((113.84090 22.21867, 113.84091 22.218..."
3,243564,"POLYGON ((113.84097 22.21871, 113.84101 22.218..."
4,243565,"POLYGON ((113.84102 22.21870, 113.84101 22.218..."
...,...,...
340747,403798,"POLYGON ((114.43641 22.53999, 114.43638 22.539..."
340748,4963,"POLYGON ((114.43640 22.54116, 114.43640 22.541..."
340749,4952,"POLYGON ((114.43668 22.54119, 114.43659 22.541..."
340750,5040,"POLYGON ((114.43675 22.54050, 114.43670 22.540..."


## 关联 building_shp 和 location_lat_lon_gdf  #merged

In [15]:
merged = gpd.sjoin(building_shp, location_lat_lon_gdf, how='inner', predicate='intersects')
# merged = merged.drop('index_right', axis=1)
merged

,BUILDINGST,geometry,index_right,device_id
310,246850,"POLYGON ((113.85992 22.25518, 113.85987 22.255...",2818977,2b4fd610-a53b-4859-a1fd-d65686bb9145
494,336293,"POLYGON ((113.86227 22.25378, 113.86222 22.253...",2584067,cb70e6e9-b2ce-6a70-becb-b807e32c87de
494,336293,"POLYGON ((113.86227 22.25378, 113.86222 22.253...",2584069,cb70e6e9-b2ce-6a70-becb-b807e32c87de
494,336293,"POLYGON ((113.86227 22.25378, 113.86222 22.253...",2584068,cb70e6e9-b2ce-6a70-becb-b807e32c87de
494,336293,"POLYGON ((113.86227 22.25378, 113.86222 22.253...",2584082,cb70e6e9-b2ce-6a70-becb-b807e32c87de
...,...,...,...,...
338798,85754,"POLYGON ((114.31586 22.39473, 114.31579 22.394...",1856116,b34c4153-d1a9-4cc7-8d1b-d912704db125
338798,85754,"POLYGON ((114.31586 22.39473, 114.31579 22.394...",1856118,b34c4153-d1a9-4cc7-8d1b-d912704db125
339443,45989,"POLYGON ((114.32853 22.45994, 114.32853 22.460...",1314852,c9b562ae-0be9-42ab-b698-5ee414dda617
339575,352677,"POLYGON ((114.33268 22.43992, 114.33276 22.439...",565178,74243407-7daa-4a8f-82dc-e8baf8f802c1


In [16]:
merged[merged['index_right'].duplicated()]

,BUILDINGST,geometry,index_right,device_id


In [17]:
device_building_loc_df = merged[['device_id','BUILDINGST']]
device_building_loc_df

,device_id,BUILDINGST
310,2b4fd610-a53b-4859-a1fd-d65686bb9145,246850
494,cb70e6e9-b2ce-6a70-becb-b807e32c87de,336293
494,cb70e6e9-b2ce-6a70-becb-b807e32c87de,336293
494,cb70e6e9-b2ce-6a70-becb-b807e32c87de,336293
494,cb70e6e9-b2ce-6a70-becb-b807e32c87de,336293
...,...,...
338798,b34c4153-d1a9-4cc7-8d1b-d912704db125,85754
338798,b34c4153-d1a9-4cc7-8d1b-d912704db125,85754
339443,c9b562ae-0be9-42ab-b698-5ee414dda617,45989
339575,74243407-7daa-4a8f-82dc-e8baf8f802c1,352677


## 推测住址

In [18]:
## 计算每个 device_id 的每个 BUILDINGST 出现的次数
counts = device_building_loc_df.groupby(['device_id','BUILDINGST']).size()
counts

device_id                             BUILDINGST
0002d2d4-b519-4255-9167-b6c7a9fa9f5b  14860          4
000c8b3f-fc84-63b7-13b4-a674302c34b7  166010         1
0014bb13-b820-4646-accb-08f9877d4c9c  42988          1
001ba349-ab7b-4c7c-b796-fc3bd6751660  158747         2
0020bd39-a85b-489f-825c-ff9d6b0cf92a  45517          1
                                                    ..
ffcbbc9d-53c4-4228-a22c-673ace185c0f  140283         1
ffcf767c-1eaa-44eb-b6ae-3b74f6030fb6  126178         2
ffdb8998-0ca7-48e2-9fa3-339de0094167  142820         1
fff69326-c5b2-4e65-a83f-38cc351d87ba  345867         1
fffc482d-4e69-48b6-a765-9c4468f60d90  181409        15
Length: 7862, dtype: int64

In [19]:
## 保存 device_id 和 这个BUILDINGST出现次数最多的 BUILDINGST
max_BUILDINGST = counts.groupby('device_id').idxmax().apply(lambda x: x[1])
max_BUILDINGST

device_id
0002d2d4-b519-4255-9167-b6c7a9fa9f5b     14860
000c8b3f-fc84-63b7-13b4-a674302c34b7    166010
0014bb13-b820-4646-accb-08f9877d4c9c     42988
001ba349-ab7b-4c7c-b796-fc3bd6751660    158747
0020bd39-a85b-489f-825c-ff9d6b0cf92a     45517
                                         ...  
ffcbbc9d-53c4-4228-a22c-673ace185c0f    140283
ffcf767c-1eaa-44eb-b6ae-3b74f6030fb6    126178
ffdb8998-0ca7-48e2-9fa3-339de0094167    142820
fff69326-c5b2-4e65-a83f-38cc351d87ba    345867
fffc482d-4e69-48b6-a765-9c4468f60d90    181409
Length: 6085, dtype: int64

In [20]:
# f_counts = counts[counts > 1] ## 剔除只出现过一次的building
# f_counts.groupby('device_id').idxmax().apply(lambda x: x[1])

device_id
0002d2d4-b519-4255-9167-b6c7a9fa9f5b     14860
001ba349-ab7b-4c7c-b796-fc3bd6751660    158747
003a7783-07cc-6bd9-29a7-2c4484e7e8d2     42418
003f6193-d195-4b1d-9e56-323a07e3ea65    220634
0081143a-d9c7-40be-9d8c-28ada9767c60     15063
                                         ...  
ff07b1b6-bee5-47ef-b6fb-f1c4f6cf11d7    138664
ffa4735f-9fee-40c5-a454-ddd7d28ca99e    109696
ffbc7091-e81a-46a2-ae95-074d64faee4f    393386
ffcf767c-1eaa-44eb-b6ae-3b74f6030fb6    126178
fffc482d-4e69-48b6-a765-9c4468f60d90    181409
Length: 2715, dtype: int64

In [21]:
location_building_pred_df = pd.DataFrame({'device_id': max_BUILDINGST.index, 'BUILDINGST': max_BUILDINGST.values})
location_building_pred_df

## 44258 个用户中， 成功推测出住址的有 6085 个

,device_id,BUILDINGST
0,0002d2d4-b519-4255-9167-b6c7a9fa9f5b,14860
1,000c8b3f-fc84-63b7-13b4-a674302c34b7,166010
2,0014bb13-b820-4646-accb-08f9877d4c9c,42988
3,001ba349-ab7b-4c7c-b796-fc3bd6751660,158747
4,0020bd39-a85b-489f-825c-ff9d6b0cf92a,45517
...,...,...
6080,ffcbbc9d-53c4-4228-a22c-673ace185c0f,140283
6081,ffcf767c-1eaa-44eb-b6ae-3b74f6030fb6,126178
6082,ffdb8998-0ca7-48e2-9fa3-339de0094167,142820
6083,fff69326-c5b2-4e65-a83f-38cc351d87ba,345867


In [22]:
## 保存数据：device_building_location
location_building_pred_df.to_csv(path + "dataset3/device_building_location.csv", index=False)